## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,31.06,95,100,10.92,overcast clouds
1,1,Kashi,CN,39.4547,75.9797,74.41,31,100,20.13,moderate rain
2,2,Bethel,US,41.3712,-73.4140,74.10,84,100,3.44,overcast clouds
3,3,Ngunguru,NZ,-35.6167,174.5000,61.57,70,100,20.47,overcast clouds
4,4,Bathsheba,BB,13.2167,-59.5167,83.01,74,40,16.11,scattered clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bathsheba,BB,13.2167,-59.5167,83.01,74,40,16.11,scattered clouds
6,6,Kathu,TH,7.9178,98.3332,85.75,82,100,17.74,moderate rain
8,8,Almeria,ES,37.1667,-2.3333,82.44,23,32,8.32,scattered clouds
10,10,Georgetown,MY,5.4112,100.3354,84.15,79,20,1.14,few clouds
15,15,Puerto Ayora,EC,-0.7393,-90.3518,76.96,90,99,11.01,overcast clouds
16,16,Sorong,ID,-0.8833,131.2500,79.43,79,76,1.19,broken clouds
21,21,Moerai,PF,-22.4333,-151.3333,75.85,79,91,12.44,light rain
26,26,Hithadhoo,MV,-0.6000,73.0833,84.27,70,33,10.29,scattered clouds
29,29,Touros,BR,-5.1989,-35.4608,82.69,67,98,9.28,overcast clouds
35,35,Rikitea,PF,-23.1203,-134.9692,76.24,72,2,8.41,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,
6,Kathu,TH,85.75,moderate rain,7.9178,98.3332,
8,Almeria,ES,82.44,scattered clouds,37.1667,-2.3333,
10,Georgetown,MY,84.15,few clouds,5.4112,100.3354,
15,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
16,Sorong,ID,79.43,broken clouds,-0.8833,131.2500,
21,Moerai,PF,75.85,light rain,-22.4333,-151.3333,
26,Hithadhoo,MV,84.27,scattered clouds,-0.6000,73.0833,
29,Touros,BR,82.69,overcast clouds,-5.1989,-35.4608,
35,Rikitea,PF,76.24,clear sky,-23.1203,-134.9692,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,Atlantis Hotel
6,Kathu,TH,85.75,moderate rain,7.9178,98.3332,Novotel Phuket Resort
8,Almeria,ES,82.44,scattered clouds,37.1667,-2.3333,Casa Rural El Laurel
10,Georgetown,MY,84.15,few clouds,5.4112,100.3354,Cititel Penang
15,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Sorong,ID,79.43,broken clouds,-0.8833,131.2500,Waigo Splash Hotel
21,Moerai,PF,75.85,light rain,-22.4333,-151.3333,Le Manotel
26,Hithadhoo,MV,84.27,scattered clouds,-0.6000,73.0833,Scoop Guest House
29,Touros,BR,82.69,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
35,Rikitea,PF,76.24,clear sky,-23.1203,-134.9692,People ThankYou


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))